In [ ]:
pip install geopy

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
import pandas as pd
from geopy.geocoders import Nominatim
import json

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "client-id",
"fs.azure.account.oauth2.client.secret": 'client-secretkey',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d8e4ef1e-4328-4930-889a-1cadcedb3365/oauth2/token"}

dbutils.fs.mount(
source = "abfss://footballdataanalytics@footballdataanaltics.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/football",
extra_configs = configs)

True

In [ ]:
%fs
ls "/mnt/football/"

path,name,size,modificationTime
dbfs:/mnt/football/raw_data/,raw_data/,0,1700258195000
dbfs:/mnt/football/transformed_data/,transformed_data/,0,1700344155000


In [ ]:
blob_storage_path = "/dbfs/mnt/football/raw_data/stadium_raw_20231118_014843.csv"
df = pd.read_csv(blob_storage_path)

In [ ]:
df

,rank,stadium,capacity,region,country,city,images,home_team
0,1,Rungrado 1st of May Stadium,114000,East Asia,North Korea,Pyongyang,https://upload.wikimedia.org/wikipedia/commons...,"Korea DPR national football team, Korea DPR wo..."
1,2,Michigan Stadium,107601,North America,United States,"Ann Arbor, Michigan",https://upload.wikimedia.org/wikipedia/commons...,Michigan Wolverines football
2,3,Ohio Stadium,102780,North America,United States,"Columbus, Ohio",https://upload.wikimedia.org/wikipedia/commons...,Ohio State Buckeyes football
3,4,Melbourne Cricket Ground,100024,Oceania,Australia,"Melbourne, Victoria",https://upload.wikimedia.org/wikipedia/commons...,"Australia national cricket team, Victoria cric..."
4,5,Camp Nou,99354,Europe,Spain,"Barcelona, Catalonia",https://upload.wikimedia.org/wikipedia/commons...,FC Barcelona
...,...,...,...,...,...,...,...,...
376,377,Bao'an Stadium,40000,East Asia,China,"Shenzhen, Guangdong",https://upload.wikimedia.org/wikipedia/commons...,NaN
377,378,Kunming Tuodong Sports Centre Stadium,40000,East Asia,China,"Kunming, Yunnan",https://upload.wikimedia.org/wikipedia/commons...,China national football team
378,379,Taizhou Sports Centre Stadium,40000,East Asia,China,"Taizhou, Zhejiang",https://upload.wikimedia.org/wikipedia/commons...,NaN
379,380,Wuhu Olympic Stadium,40000,East Asia,China,"Wuhu, Anhui",https://upload.wikimedia.org/wikipedia/commons...,NaN


In [ ]:
import pandas as pd
from geopy.geocoders import Photon

import json


NO_IMAGE = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/No-image-available.png/480px-No-image-available.png'

def get_lat_long(country, city_or_stadium):
    geolocator = Photon(user_agent="measurements")
    location = geolocator.geocode(f'{city_or_stadium}, {country}')

    if location:
        return location.latitude, location.longitude

    return None, None

def transform_wikipedia_data(blob_storage_path):
    stadiums_df = pd.read_csv(blob_storage_path)
    stadiums_df['location'] = stadiums_df.apply(lambda x: get_lat_long(x['country'], x['stadium']), axis=1)
    stadiums_df['images'] = stadiums_df['images'].apply(lambda x: x if x not in ['NO_IMAGE', '', None] else NO_IMAGE)
    stadiums_df['capacity'] = stadiums_df['capacity'].astype(int)

    duplicates = stadiums_df[stadiums_df.duplicated(['location'], keep=False)]
    for index, row in duplicates.iterrows():
        lat, long = get_lat_long(row['country'], row['city'])
        stadiums_df.at[index, 'location'] = (lat, long)

    output_path = "/dbfs/mnt/football/transformed_data/transformed_stadium_data.csv"
    stadiums_df.to_csv(output_path, index=False)

    return "OK"

transform_wikipedia_data('/dbfs/mnt/football/raw_data/stadium_raw_20231118_014843.csv')


'OK'